# Image labelling - Graphical User Interface (GUI)

This notebook implements a `Graphical User Interface (GUI)` to manually label the images. <br>
The GUI automatically parses the `png_unfiltered` folder and extracts all the images. It plots on the left the original `RAW` image. On the right, it shows a `Filtered` version to help you to visualize the image. Filtering is applied through the `equalize_tensor` algorithm implemented in [PyRawS](https://github.com/ESA-PhiLab/PyRawS/blob/main/pyraws/utils/visualization_utils.py). 

## 1. - Imports

In [16]:
%matplotlib widget
%matplotlib inline
%load_ext autoreload
%autoreload 2
# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import torch
from torchvision.io import read_image
import os
import sys
sys.path.insert(1, os.path.join("..", "data"))
sys.path.insert(1, os.path.join("..", "utils"))
import matplotlib.pyplot as plt
import numpy as np
from plot_utils import plot_image
from glob import glob
import ipywidgets as widgets
from IPython.display import display, clear_output
import shutil


## 2. - Creating directories and pointing to their path

In [18]:
data_folder=os.path.join("..", "data")
cloud_folder=os.path.join(data_folder, "Cloud")
good_folder=os.path.join(data_folder, "Good")
edge_folder=os.path.join(data_folder, "Edge")
png_folder=os.path.join(data_folder, "png_unfiltered")

# Creating folders if not existing
os.makedirs(cloud_folder, exist_ok=True)
os.makedirs(good_folder, exist_ok=True)
os.makedirs(edge_folder, exist_ok=True)

## 3. - Running GUI

### GUI Description

The GUI shows four buttons on the top: <br>
- `"_"`, corresponding to unknown. 
- `Cloud`, corresponding to the class `Cloud`.
- `Edge`, corresponding to the class `Egde`.
- `Good`, corresponding to the class `Good`.

At the center, the GUI shows the `Raw` and the `Filtered` image. 
On the bottom, you can see two buttons `Prev` and `Next`, which makes you select respectively the previous or the next image in the list. <br>

### How to use the GUI

Proceed as follows:
1. Selects the corresponding class by pressing one of the buttons on top of the GUI. Select `_` if you are not sure.
2. Press `Next` or `Prev` buttons to move respectively upward or backward.
3. If the class that you selected `_`, the image is just skipped and can be reprocessed again, simply pressing the button that makes you parse the list in the opposite direction than before.
4. If the class selected is **NOT** `_`, then the image is moved from the `png_folder` to the correspondent folder (e.g., `Good` folder if you had selected `Good`) and will be removed from the list of the images to be processed. 

In [19]:
#Change this to speed up the processing
figsize=(10,20)
downsampling_factor=(2,2)

In [20]:
plt.ioff()

# Read images in PNG_FOLDER
images_path=sorted(glob(os.path.join(png_folder, "*")))
# Setting all the pointers to 0
n_pointer=0
n_images=len(images_path)
# Read first image
if n_images != 0:
    image=read_image(images_path[0])


# Get output
output = widgets.Output()
if n_images != 0:
    # Plotting first image
    with output:
        clear_output(wait=True)
        ax=plot_image(image, figsize=figsize,downsampling_factor=downsampling_factor)
        display(ax.figure)
else:
    with output:
        print("Processing finished")
# Get widgets
next = widgets.Button(description='Next')
prev = widgets.Button(description='Prev')
buttons=widgets.HBox([prev, next])
toggle_buttons = widgets.ToggleButtons(
    options=['_', 'Cloud', 'Edge', 'Good'],
    description='Output dir:',
)
# Get directory path depending on directory
def get_directory(directory):
    """
    Print the current widget value in short sentence
    """
    if directory == "_":
        selected_directory=""
    elif directory == "Cloud":
        selected_directory=cloud_folder
    elif directory == "Good":
        selected_directory=good_folder
    elif directory == "Edge":
        selected_directory=edge_folder
    else:
        selected_directory=""
    return selected_directory

# Function getter for images_path
def get_images_path():
    global images_path
    return images_path
# Function setter for images_path
def set_images_path(images_path_new):
    global images_path
    images_path = images_path_new

# Function getter for n_pointer
def get_n_pointer():
    global n_pointer
    return n_pointer
# Function setter for n_pointer
def set_n_pointer(n_pointer_new):
    global n_pointer
    n_pointer = n_pointer_new

# Setting up widgets
vbox=widgets.VBox([toggle_buttons, output, buttons])
display(vbox)

# Handler for next button
def move_next(obj):
    # Getters
    n_pointer = get_n_pointer()
    images_path = get_images_path()
    # Holding old values for directory path
    path_old=images_path[n_pointer]
    # Getting directory
    selected_directory=get_directory(toggle_buttons.value)

    # Increment pointer with wrap around to 0
    n_pointer = (n_pointer + 1) %  len(images_path)
    # Read new images
    image=read_image(images_path[n_pointer])
    # Set n_pointer
    set_n_pointer(n_pointer)
    # Plot new images
    if (len(images_path) > 1) or (len(images_path) == 1 and selected_directory==""):
        with output:
            clear_output(wait=True)
            ax=plot_image(image,figsize=figsize,downsampling_factor=downsampling_factor)
            display(ax.figure)
    else:
        clear_output(wait=True)
    # Calculating new path
    path_new=os.path.join(selected_directory, path_old.split(os.sep)[-1])
    # Moving image and removing image from png_path
    if selected_directory != "":
        shutil.move(path_old,path_new)
        images_path.remove(path_old)
        set_images_path(images_path)
        if n_pointer == len(images_path):
            set_n_pointer(0)

    if len(images_path) == 0:
        with output:
            print("Processing finished")

# Handler for prev button
def move_prev(obj):
    # Getters
    n_pointer = get_n_pointer()
    images_path = get_images_path()
    # Holding old values for directory path
    path_old=images_path[n_pointer]
    # Getting directory
    selected_directory=get_directory(toggle_buttons.value)

    # Decreasing n_pointer with wrap around to n_images - 1
    n_pointer = n_pointer - 1
    if n_pointer <= 0:
        n_pointer =  len(images_path) - 1
    # Read new images
    image=read_image(images_path[n_pointer])
    # Set n_pointer
    set_n_pointer(n_pointer)
    if (len(images_path) > 1) or (len(images_path) == 1 and selected_directory==""):
        # Plot new images
        with output:
            clear_output(wait=True)
            ax=plot_image(image,figsize=figsize,downsampling_factor=downsampling_factor)
            display(ax.figure)
    else:
        clear_output(wait=True)

    # Calculating new path
    path_new=os.path.join(selected_directory, path_old.split(os.sep)[-1])
    # Moving image and removing image from png_path
    if selected_directory != "":
        shutil.move(path_old,path_new)
        images_path.remove(path_old)
        set_images_path(images_path)
        if n_pointer == len(images_path):
            set_n_pointer(0)

    if len(images_path) == 0:
        with output:
            print("Processing finished")

# Setting handlers
next.on_click(move_next)
prev.on_click(move_prev)